In [1]:
## note that active directory is already set in thfunction_file_news_corpus
## Keep all files in the same active dir
## refer to line 48 and line 49 of the function file
## pip install --upgrade gensim ( if not installed already )
## pip install lda (if not installed already )
## pip install pyldavis (if not installed already )
## pip install tqdm (if not installed already )
## conda install scikit-learn (if not installed already )
## LDA is built on top of gensim

from function_file_news_corpus import *

In [ ]:
## apply LDA to list of num topics = [5, 10, 15, 20,21, 22, 23, 24, 25 , 26 , 27, 28,29, 30, 32, 34, 36, 38, 40]
## store results and check for topics for each
## dict dfs stores the topic file for each num topic and also storesa data frame Lda_df, that is a replica
## of the original data frame with a new column called 'topic' 
## with a range [0 : num_topic] for num_topic in count_topics
## set _top_topic words as n_top_words

news_df = pd.read_pickle('final_clean_hindu_corpus.pkl')
news_df.drop(['DateTime'], axis = 1, inplace = True)


from sklearn.feature_extraction.text import CountVectorizer
cvectorizer = CountVectorizer(max_df = 0.4, min_df=0.01, max_features=10000, tokenizer=tokenizer_tf_idf,
                              stop_words= stopwords(), ngram_range=(1,1))
cvz = cvectorizer.fit_transform(news_df['text'])
vocab = cvectorizer.get_feature_names()

import lda
import logging
import pyLDAvis

logging.getLogger("lda").setLevel(logging.WARNING)

count_topics = [5, 10, 15, 20,21, 22, 23, 24, 25 , 26 , 27, 28,29, 30, 32, 34, 36, 38, 40]
dict_dfs = []
list_dfs = []
for n_topics in count_topics:
    name_0 = './pyldadavis_v7' + str(n_topics) 
    name = name_0 + '.html'
    name_csv = name_0 +'.csv'
    print('n_topics = ', n_topics )
    n_iter = 3000
    lda_model = lda.LDA(n_topics=n_topics, n_iter=n_iter)
    X_topics = lda_model.fit_transform(cvz)
    n_top_words = 20
    topic_summaries = []

    topic_word = lda_model.topic_word_  # get the topic words
    vocab = cvectorizer.get_feature_names()

    topic_file = []

    for i, topic_dist in enumerate(topic_word):
        topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
        topic_summaries.append(' '.join(topic_words))
        top = 'Topic {}: {}'.format(i, ' '.join(topic_words))
        topic_file.append(dict(topic = i, topic_name = top))

    topic_file = pd.DataFrame(topic_file)

    topic_file.to_csv(name_csv)

    from sklearn.manifold import TSNE
    tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
    tsne_lda = tsne_model.fit_transform(X_topics)
    doc_topic = lda_model.doc_topic_
    lda_keys = []

    for i, tweet in enumerate(news_df['text']):
        lda_keys += [doc_topic[i].argmax()]


    lda_df = pd.DataFrame(tsne_lda, columns=['x','y'])
    lda_df['doct_no'] = news_df['doct_no']
    lda_df['topic'] = lda_keys
    lda_df['text'] = news_df['text']
    lda_df['original_text'] = news_df['original_text']
    lda_df['date'] = news_df['num_date']

    #Preparing Lda_df
    news_df['tokens'] = news_df['text'].map(tokenizer_tf_idf)
    lda_df['len_docs'] = news_df['tokens'].map(len)
    list_dfs.append(lda_df)
    dict_dfs.append(dict(num_topic =n_topics,  Lda_df = lda_df, topics = topic_file))

    def prepareLDAData():
        data = {
            'vocab': vocab,
            'doc_topic_dists': lda_model.doc_topic_,
            'doc_lengths': list(lda_df['len_docs']),
            'term_frequency':cvectorizer.vocabulary_,
            'topic_term_dists': lda_model.components_
        } 
        return data
    ldadata = prepareLDAData()

    import pyLDAvis

    pyLDAvis.enable_notebook()
    prepared_data = pyLDAvis.prepare(**ldadata)
    pyLDAvis.save_html(prepared_data,name)


    print('############################################')



In [ ]:
## run LDA again specifically for num topics = 29 
## post results in the file given below in the active directory 


news_df = pd.read_pickle('final_clean_hindu_corpus.pkl')
news_df.drop(['DateTime'], axis = 1, inplace = True)


news_df.head()

from sklearn.feature_extraction.text import CountVectorizer
cvectorizer = CountVectorizer(max_df = 0.4, min_df=0.01, max_features=10000, tokenizer=tokenizer_tf_idf,
                              stop_words= stopwords(), ngram_range=(1,1))
cvz = cvectorizer.fit_transform(news_df['text'])
vocab = cvectorizer.get_feature_names()

import lda
import logging
logging.getLogger("lda").setLevel(logging.WARNING)

count_topics = [29]
dict_dfs = []
list_dfs = []
for n_topics in count_topics:
    name_0 = './pyldadavis_v7' + str(n_topics) 
    name = name_0 + '.html'
    name_csv = name_0 +'.csv'
    print('n_topics = ', n_topics )
    n_iter = 3000
    lda_model = lda.LDA(n_topics=n_topics, n_iter=n_iter)
    X_topics = lda_model.fit_transform(cvz)
    n_top_words = 20
    topic_summaries = []

    topic_word = lda_model.topic_word_  # get the topic words
    vocab = cvectorizer.get_feature_names()

    topic_file = []

    for i, topic_dist in enumerate(topic_word):
        topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
        topic_summaries.append(' '.join(topic_words))
        top = 'Topic {}: {}'.format(i, ' '.join(topic_words))
        topic_file.append(dict(topic = i, topic_name = top))

    topic_file = pd.DataFrame(topic_file)

    topic_file.to_csv(name_csv)

    from sklearn.manifold import TSNE
    tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
    tsne_lda = tsne_model.fit_transform(X_topics)
    doc_topic = lda_model.doc_topic_
    lda_keys = []

    for i, tweet in enumerate(news_df['text']):
        lda_keys += [doc_topic[i].argmax()]


    lda_df = pd.DataFrame(tsne_lda, columns=['x','y'])
    lda_df['doct_no'] = news_df['doct_no']
    lda_df['topic'] = lda_keys
    lda_df['text'] = news_df['text']
    lda_df['original_text'] = news_df['original_text']
    lda_df['date'] = news_df['num_date']

    #Preparing Lda_df
    news_df['tokens'] = news_df['text'].map(tokenizer_tf_idf)
    lda_df['len_docs'] = news_df['tokens'].map(len)
    list_dfs.append(lda_df)
    dict_dfs.append(dict(num_topic =n_topics,  Lda_df = lda_df, topics = topic_file))

    def prepareLDAData():
        data = {
            'vocab': vocab,
            'doc_topic_dists': lda_model.doc_topic_,
            'doc_lengths': list(lda_df['len_docs']),
            'term_frequency':cvectorizer.vocabulary_,
            'topic_term_dists': lda_model.components_
        } 
        return data
    ldadata = prepareLDAData()

    import pyLDAvis

    pyLDAvis.enable_notebook()
    prepared_data = pyLDAvis.prepare(**ldadata)
    pyLDAvis.save_html(prepared_data,name)


    print('############################################')



In [23]:
## Save results of LDA with 29 topics for specific analysis

news_df_topics_for_sentiment_analysis_final = dict_dfs[0]['Lda_df'].copy(deep=True)
news_df_topics_for_sentiment_analysis_final.drop(['x', 'y' , 'len_docs'], axis = 1, inplace = True)
news_df_topics_for_sentiment_analysis_final.to_pickle('news_df_topics_for_sentiment_analysis_final.pkl')


news_df_topic_key_final = dict_dfs[0]['topics'].copy(deep=True)
writer = pd.ExcelWriter('news_df_topic_key_final.xlsx')
news_df_topic_key_final.to_excel(writer, sheet_name='topic_key')
writer.save()